# Ordering and Delivery

This notebook demonstrates ordering and download with the orders api. In this notebook, we check authentication by requesting an orders list, then we create an order for two Analytic `PSScene4Band` images. We poll for order success then download images individually. And finally, we create, poll, and download the same order delivered as a single zip file.

Reference information can be found at [Ordering & Delivery](https://developers.planet.com/docs/orders/ordering-delivery/).

In [ ]:
import json
import os
import pathlib
import time

import requests
from requests.auth import HTTPBasicAuth

## Authenticating

In [ ]:
# API Key stored as an env variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

In [ ]:
orders_url = 'https://api.planet.com/compute/ops/orders/v2/'
data_url = "https://api.planet.com/data/v1"

### Curl example

To check your orders list and make sure you have the permissions you need, uncomment the following line to run `curl`

In [ ]:
#!curl -L -H "Authorization: api-key $PLANET_API_KEY" $orders_url

In [ ]:
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(data_url, auth=auth)
response

## Searching with the Data API
We can use the [data API](https://developers.planet.com/docs/apis/data/) in order to automate searching based on the search criterias like: date range, cloud cover, area cover, aoi. We can create and AOI using https://geojson.io/

In [ ]:
geometry = {
    "type":"Polygon",
    "coordinates":[
          [
            [
              -119.97872882829992,
              35.31103603238796
            ],
            [
              -119.97872882829992,
              34.98939831283745
            ],
            [
              -119.48645226655668,
              34.98939831283745
            ],
            [
              -119.48645226655668,
              35.31103603238796
            ],
            [
              -119.97872882829992,
              35.31103603238796
            ]
          ]
        ]
}

In [ ]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte":"2023-03-29T00:00:00Z",
    "lte":"2023-04-10T00:00:00Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.1
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [ ]:
item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)

# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]
print(image_ids)

### Requests example

In this notebook, we will be using `requests` to communicate with the orders v2 API. First, we will check our orders list to make sure authentication and communication is working as expected.

We want to get a response code of `200` from this API call. To troubleshoot other response codes, see the [List Orders](https://developers.planet.com/docs/orders/reference/#operation/listOrders) AOI reference.

In [ ]:
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

## Ordering

In this example, we will order two `PSScene4Band` analytic images. For variations on this kind of order, see [Ordering Data](https://developers.planet.com/docs/orders/ordering-delivery/#ordering-data_1).

In this order, we request an `analytic` bundle. A bundle is a group of assets for an item. The `analytic` bundle for the  `PSScene4Band` item contains 3 assets: the analytic image, the analytic xml file, and the udm. See the [Product bundles reference](https://developers.planet.com/docs/orders/product-bundles-reference/) to learn about other bundles and other items.

Now we will list the names of orders we have created thus far. Your list may be empty if you have not created an order yet.

In [ ]:
orders = response.json()['orders']
[r['name'] for r in orders[:5]]

### Place Order

In [ ]:
# set content type to json
headers = {'content-type': 'application/json'}

In [ ]:
# define products part of order
single_product = [
    {
      "item_ids": ["20220628_183020_20_248c"],
      "item_type": "PSScene",
      "product_bundle": "ortho_analytic"
    }
]

same_src_products = [
    {
      "item_ids": ["20151119_025740_0c74",
                   "20151119_025739_0c74"],
      "item_type": "PSScene",
      "product_bundle": "orhto_analytic"
    }
]

multi_src_products = [
    {
      "item_ids": ["20151119_025740_0c74"],
      "item_type": "PSScene",
      "product_bundle": "ortho_analytic"
    },
    {
      "item_ids": ["20220628_183020_20_248c"],
      "item_type": "PSScene",
      "product_bundle": "ortho_analytic"
    },
    
]

In [ ]:
product = [
    {
      "item_ids": image_ids[:2],
      "item_type": "PSScene",
      "product_bundle": "visual"
    }
]


request = {  
   "name":"Super Bloom image",
   "products": product,
    "delivery": {"single_archive": True, "archive_type": "zip"}
}

In [ ]:
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response.json())
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

In [ ]:
order_url = place_order(request, auth)

### Poll for Order Success

In [ ]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(5)
        
poll_for_success(order_url, auth)

## Bandmath and clip
Now lets use the toolchain in order to combine a few opperations at once.

To create a polygon to clip by lets go to [geojson.io](https://geojson.io)


In [ ]:
# define the clip tool
clip = {
    "clip": {
        "aoi": geometry
    }
}


In [ ]:
 composite = {"composite": {}}

In [ ]:
bandmath = {
  "bandmath": {
    "b1": "b1",
    "b2": "b2",
    "b3": "b2",
#     "b4": "b4",
#     "b5": "b5",
#     "b6": "b6",
#     "b7": "b7",
#     "b8": "b8",
#     "b9": "(b8 - b6) / (b8 + b6)",
#     "pixel_type": "32R",
  }
}

In [ ]:
tool_request = { 
    "name":"Salton Sea Clipped",
    "products": product,
    "tools": [clip, bandmath, composite],
    "delivery": {"single_archive": True, "archive_type": "zip"}
}

In [ ]:
tool_order_url = place_order(tool_request, auth)

### View Results
Now lets review our previous order and download it

In [ ]:
requests.get(order_url, auth=auth).json()['state']

In [ ]:
r = requests.get(order_url, auth=auth)
response = r.json()
results = response['_links']['results']

In [ ]:
[r['name'] for r in results]

## Download

### Downloading each asset individually

In [ ]:
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))

In [ ]:
download_results(results)

## Basemaps

In [ ]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'
auth = HTTPBasicAuth(PLANET_API_KEY, '')
basemapServiceResponse = requests.get(url=BASEMAP_API_URL, auth=auth)

In [ ]:
basemaps = basemapServiceResponse.raise_for_status()
if basemapServiceResponse.status_code != 204:
    basemaps = json.loads(basemapServiceResponse.text)
mosaics = basemaps['mosaics']
names = [feature['name'] for feature in mosaics]
print(names)

In [ ]:
# set content type to json
headers = {'content-type': 'application/json'}

In [ ]:
order_params = {
   "name": "Basemap order with geometry",
   "source_type": "basemaps",
   "products": [
      {
         "mosaic_name": "global_monthly_2023_01_mosaic",
         "geometry":geometry
      }
   ],
    "tools": [clip],
}

In [ ]:
basemap_url = place_order(order_params, auth)